In [ ]:
import cv2
import holoviews as hv
from holoviews import opts
import param
import panel as pn
import numpy as np
import holoviews as hv
from holoviews import opts
import numpy as np
import pandas as pd
import utils as rad
from pathlib import Path
pn.extension()
hv.extension('bokeh')
import logging
logging.basicConfig(format='%(asctime)s %(name)s %(levelname)s:%(message)s', level=logging.DEBUG)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# CONSTANTS
VISUAL_WIDTH = 800
VISUAL_HEIGHT = 800

def visualize(graph):
    return graph.opts(width=VISUAL_WIDTH, height=VISUAL_HEIGHT)

# Data Loading

Use this cell to point to the location of your 3 sources
1. MRI source (this the folder path)
1. Pathology source (this is the image path)
1. Pathology annotated image source (this is the annotated image path)

In [ ]:
# FILL IN YOUR VALUES HERE IF YOU DON'T WANT TO USE THE SAMPLE
mri_folder_path = f'data/patient87/Prostate Fused-MRI-Pathology/aaa0087/09-01-2000-PELVISPROSTATE-81724/5.000000-T2 AXIAL SM FOV-30095'
pathology_img_path = f'data/patient87/pathology/aaa0087 F2F3F1F4.tif'
pathology_annotated_img_path = 'data/patient87/pathology/masks/aaa0087 F2F3F1F4.trace.tif'

print("pathology_img_path exists:", Path(pathology_img_path).exists())
print("pathology_annotated_img_path exists:", Path(pathology_annotated_img_path).exists())

**Warning**: This might take a while depening on the size of the image

In [ ]:
pathology_source = rad.Image.load(pathology_img_path)
pathology_annotated_source = rad.Image.load(pathology_annotated_img_path)

# Select and Upscale the MRI slice

Use the follwing cell to select the slice of the MRI image. Additionally, select the upscaled ascpet ratio of the MRI image.

In [ ]:
mri_viewer = rad.MriVisualizer(mri_path=mri_folder_path)
mri_viewer.panel()

# Downscale the Pathology image

In the following cell, downscale the pathology image. Similarly, there are options to transform the image.

Note: This is normally a very large image file. Downscaling this image as small as possible to find the orientation.
      
**WARNING**: The aspect ratio is of the original image at angle 0. Not the rotated bounds of the chart that are visually displayed.

In [ ]:
pathology_viewer = rad.PathologyVisualizer(pathology_img=pathology_source)
pathology_viewer.panel()

# Annotation Overlay Validation

The below visualization is to show the annotated pathology image along with the pathology image.

In [ ]:
pathology_viewer.side_by_side(pathology_annotated_source)

# Landmark Selection

After completing the upscaling and downscaling of both the mri image and pathology image. We will use the next control to select the landmarks.

Use the Point Draw tool to select the points. Refrence to point draw selection can be found here: https://holoviews.org/reference/streams/bokeh/PointDraw.html

Note: You can zoom and pan by using the controls.

**WARNING**: Select the same amount of points in both images. The points are associated by color. A new point will always be colored white upon creation and will be changed to the default color if the context changes.

**WARNING**: If you need to reselect the MRI slice or reorient the pathology image, you can go above and update the values in the above control. However, for it to take effect, you will have rerun the following cell.

### Using Existing Registration

In this cell, we will load existing configurations (landmarks and transformations) if they exists and updated the viewer with the cached data. **If you don't want to use existing data from a cache, either update the cell to files that do not exist or do not run the cell.**

In [ ]:
cached_mri_landmarks_path = f"data/output/fixed.json"
cached_pathology_landmarks_path = f"data/output/moving.json"

fixed_img = mri_viewer.fixed_image.load_data(cached_mri_landmarks_path)
moving_img = pathology_viewer.moving_image.load_data(cached_pathology_landmarks_path)

In [ ]:
landmarks_viewer = rad.LandmarkSelectorVisualizer(fixed_img=fixed_img, moving_img=moving_img)
landmarks_viewer.panel()

# Landmark association verification

The below cell is a table verifying the associated points selected from the pervious cell.

**Note**: The origin(reference frame) is bottom left. Depending on the library you are using, there are different conventions.

In [ ]:
landmarks_viewer.associated_points_to_DF()

In [ ]:
tps_viewer = rad.TpsAlgoVisualizer(
    fixed_image = landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT),
    moving_image = landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)
)


# Intermediate landmark visualization

The following 6 plots show the intermediate steps of both the landmarks and the images:

1. Visualize only the MRI and the landmarks
2. Visualize only the pathology and the affined landmarks
3. Visualize only the Thin Plate Splines and the moved landmarks
4. Visualize both the MRI and the affined Pathology image and the Pathology affined landmarks
5. Visualize both the MRI and the Thin Plate Splines Pathology image and the Pathology Thin Plate Spline landmarks
6. Visualize several components: the MRI, the Thin Plate Splin Pathology,the Pathology affined landmarks and the Pathology Thin Plate Spline landmarks

Finally, we will show all 6 of these charts at the same time

## 1. Visualize only the MRI and the landmarks

In [ ]:
visualize(tps_viewer.mri_with_fixed_points())

## 2. Visualize only the pathology and the affined landmarks

In [ ]:
visualize(tps_viewer.pathology_with_affine_points())

## 3. Visualize only the Thin Plate Splines and the moved landmarks

In [ ]:
visualize(tps_viewer.pathology_with_tps_points())

## 4. Visualize the MRI, the affined Pathology image and the Pathology affined landmarks

In [ ]:
visualize(tps_viewer.mri_with_affine_overlay())

## 5. Visualize the MRI, the Thin Plate Splines Pathology image and the Pathology Thin Plate Spline landmarks

In [ ]:
visualize(tps_viewer.mri_with_tps_overlay())

## 6. Visualize several components: the MRI, the Thin Plate Spline Pathology image, the Pathology affined landmarks and the Pathology Thin Plate Spline landmarks

In [ ]:
visualize(tps_viewer.mri_with_affine_and_tps_overlay())

In [ ]:
tps_viewer.panel()

# Image Alignment Visualization

The below visualization shows the final transformations of the image registration done in the previous steps. In this visualization, you can see the pathology and mri image overlayed on top of each other. Additionally, you can control the transparency of each layer. 

**Note:** controlling the alpha might take a while for the image to update

In [ ]:
overlay_viewer = rad.TpsOverlayVisualizer(
    fixed=landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT), 
    warped=landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT), 
    annotated=pathology_annotated_source.copy_attributes(
        landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)))
overlay_viewer.panel()

# Saving the data

In this final part of the notebook, we will be saving the data so that the points can be refrenced/loaded.

Update the value in `save_path` to the destination you would like your data saved

In [ ]:
save_path = "data/output"
save_path = Path(save_path)
def save(path):
    save_path.mkdir(exist_ok=True)
    
    fixed = landmarks_viewer.fixed_image(in_origin=rad.Origin.TOP_LEFT)
    moving = landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT)
    annotated = (pathology_annotated_source.copy_attributes(landmarks_viewer.moving_image(in_origin=rad.Origin.TOP_LEFT))
                                           .update(interpolation='INTER_AREA'))
    tps_moving = rad.tps.TpsAlgorithm(moving_img=moving, fixed_img=fixed).warp()
    tps_moving_annotated = rad.tps.TpsAlgorithm(moving_img=annotated, fixed_img=fixed).warp()
    
    
    fixed.save(save_path,"fixed")
    moving.save(save_path,"moving")
    annotated.save(save_path,"annotated")
    tps_moving.save(save_path,"tps_moving")
    tps_moving_annotated.save(save_path,"tps_moving_annotated")
    
save(save_path)